# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** — ответить на следующие вопросы:

    *Есть ли зависимость между количеством детей и возвратом кредита в срок?
    *Есть ли зависимость между семейным положением и возвратом кредита в срок?
    *Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
    *Как разные цели кредита влияют на его возврат в срок?


**Описание данных**:

    *children — количество детей в семье
    *days_employed — общий трудовой стаж в днях
    *dob_years — возраст клиента в годах
    *education — уровень образования клиента
    *education_id — идентификатор уровня образования
    *family_status — семейное положение
    *family_status_id — идентификатор семейного положения
    *gender — пол клиента
    *income_type — тип занятости
    *debt — имел ли задолженность по возврату кредитов
    *total_income — ежемесячный доход
    *purpose — цель получения кредита

### Шаг 1. Обзор данных

Импортируем библиотеку Pandas.Прочитаем файл с данными для анализа и сохраним его в переменной df. Выведем на экран первые 15 строк таблицы:

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получим общую информацию о данных в таблице df:

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.1 Заполнение пропусков

В двух столбцах (days_employed и total_income) есть пропущенные значения.Пропущенные значения могли появиться по разным причинам: клиенты могли не предоставить данные, данные могли быть не внесены сотрудником банка и т.д.

Определим, сколько раз были пропущены значения в  столбцах days_employed,total_income.Определим процент пропущенных значений в столбце.

In [3]:
missing_values_days_employed = len(df[df['days_employed'].isna()]) #Кол-во пропущенных значений в столбце days_employed
missing_values_total_income = len(df[df['total_income'].isna()]) #Кол-во пропущенных значений в столбце total_income
print(f'Пропущенных значений в столбце Общий трудовой стаж: {missing_values_days_employed}, что составляет {missing_values_days_employed/21525*100:.2f}%.')
print(f'Пропущенных значений в столбце Ежемесячный доход: {missing_values_total_income}, что составляет {missing_values_total_income/21525*100:.2f}%.')

Пропущенных значений в столбце Общий трудовой стаж: 2174, что составляет 10.10%.
Пропущенных значений в столбце Ежемесячный доход: 2174, что составляет 10.10%.


В каждом столбце доля пропушенных значений превышает 10%. Строки с пропущенными значениями удалять нельзя, так как удаление такого большого количества строк может негативно отразиться на анализе данных.Заполним пропуски.

В данном случае, значения в столбцах общий трудовой стаж и ежемесячный доход - количественные переменные. Пропуски в таких переменных заполняют характерными значениями. Так как значения в столбцах сильно различаются, то среднее арифметическое  использовать некорректно. Поэтому, заполним пропуски медианными значениями.  

<b>Замена пропушенных значений в столбце days_employed.</b>

Выведем первые пять значений с пропусками столбца days_employed.

In [4]:
df[df['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Заполним пропуски, используя медианные значения, предварительно применив к значениям столбца days_employed функцию abs() для перевода всех значений в абсолютные.

In [5]:
df['days_employed'] = abs(df['days_employed']) #Перевод всех значений в абсолютные 
days_employed_mean = df['days_employed'].median() #Находим медианное значение по столбцу
df['days_employed'] = df['days_employed'].fillna(value = days_employed_mean) #Замена пропущенных значений на медианные

Выполним проверку:

In [6]:
df[df['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Пропущенные значения в столбце days_employed заменены.

<b>Замена пропушенных значений в столбце total_income.</b>

Выведем первые пять значений с пропусками столбца total_income.

In [7]:
df[df['total_income'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,2194.220567,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,2194.220567,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,2194.220567,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,2194.220567,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,2194.220567,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Заполним пропуски, используя медианные значения.

In [8]:
total_income_mean = df['total_income'].median() #Находим медианное значение по столбцу
df['total_income'] = df['total_income'].fillna(value = total_income_mean) #Замена пропущенных значений на медианные

Выполним проверку:

In [9]:
df[df['total_income'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Пропущенные значения в столбце total_income заменены.

### Шаг 2.2 Проверка данных на аномалии и исправления.

В данных  встречаются артефакты (аномалии). Данные значения появились по ошибке. 
Ранее, в шаге 2.1 отрицательное количество дней трудового стажа в столбце days_employed было исправлено с помощью функции abs(). Эта ошибка могла появиться в результате некорректной выгрузки данных, ошибок сотрудников банка при внесении информации и т.д.
Также в шаге 2.1 были заполнены все пропущенные значения.

Проверим значения в остальных столбцах таблицы на аномалии. Некорректные значения есть в столбцах children, gender и dob_years.

В столбце children есть отрицательные значения (-1), что является явной ошибкой: 

In [10]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Исправим все значения в столбце children на абсолютные. Также, предположим что значение 20 - ошибочно. Исправим данное значение на 2.

In [11]:
#Исправление всех значений на абсолютные
df['children'] = abs(df['children']) 
#Замена значений 20 в столбце children  на 2
df.loc[(df['children'] == 20), ['children']] = 2

In [12]:
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

В столбце gender есть недопустимое значение XNA.

In [13]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [14]:
rows = df['gender'] == 'XNA'
df.loc[rows]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости


Заменим значение XNA в таблице на наиболее популярный пол.

In [15]:
df.loc[(df['gender'] == 'XNA'), ['gender']] = 'F'
df['gender'].value_counts()

F    14237
M     7288
Name: gender, dtype: int64

В столбце dob_years есть нулевые значения, что является ошибкой. Заменим данное значение на медианное.

In [16]:
df['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [17]:
#Находим медианное значение
dob_years_mean = df['dob_years'].median()
#Замена нулевых значений на медианные
df.loc[(df['dob_years'] == 0), ['dob_years']] = dob_years_mean
#Проверка
df['dob_years'].value_counts()

42.0    698
35.0    617
40.0    609
41.0    607
34.0    603
38.0    598
33.0    581
39.0    573
31.0    560
36.0    555
44.0    547
29.0    545
30.0    540
48.0    538
37.0    537
50.0    514
43.0    513
32.0    510
49.0    508
28.0    503
45.0    497
27.0    493
56.0    487
52.0    484
47.0    480
54.0    479
46.0    475
58.0    461
57.0    460
53.0    459
51.0    448
59.0    444
55.0    443
26.0    408
60.0    377
25.0    357
61.0    355
62.0    352
63.0    269
64.0    265
24.0    264
23.0    254
65.0    194
66.0    183
22.0    183
67.0    167
21.0    111
68.0     99
69.0     85
70.0     65
71.0     58
20.0     51
72.0     33
19.0     14
73.0      8
74.0      6
75.0      1
Name: dob_years, dtype: int64

Также, в столбце days_employed есть некорректные значения, например общий трудовой стаж в днях равный 340266.072047 что равно более 900 лет. Данные значения скорректировать нет возможности, на анализ они не повлияют Оставим из без изменений.

### Шаг 2.3. Изменение типов данных.

Заменим вещественный тип данных в столбце total_income и days_employed на целочисленный с помощью метода astype().

In [18]:
df['total_income'] = df['total_income'].astype('int')
df['days_employed'] = df['days_employed'].astype('int')

Выполним проверку. 

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

Приведем значения в столбце education к одному регистру. 

In [20]:
df['education'] = df['education'].str.lower()

Проверим данные на наличие явных дубликатов.

In [21]:
df.duplicated().sum()

72

Удалим явные дубликаты.

In [22]:
df = df.drop_duplicates().reset_index(drop=True) 

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим  два новых датафрейма.

1. Каждому уникальному значению из education соответствует уникальное значение education_id.

In [23]:
education_df = df[['education','education_id']]
print(education_df.head(10))

  education  education_id
0    высшее             0
1   среднее             1
2   среднее             1
3   среднее             1
4   среднее             1
5    высшее             0
6    высшее             0
7   среднее             1
8    высшее             0
9   среднее             1


Удалим дубликаты.

In [24]:
education_df = education_df.drop_duplicates().reset_index(drop=True)
print(education_df.head(10)) 

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4


2. Каждому уникальному значению из family_status соответствует уникальное значение family_status_id.


In [25]:
family_df = df[['family_status','family_status_id']]
print(family_df.head(10))

      family_status  family_status_id
0   женат / замужем                 0
1   женат / замужем                 0
2   женат / замужем                 0
3   женат / замужем                 0
4  гражданский брак                 1
5  гражданский брак                 1
6   женат / замужем                 0
7   женат / замужем                 0
8  гражданский брак                 1
9   женат / замужем                 0


Удалим дубликаты.

In [26]:
family_df = family_df.drop_duplicates().reset_index(drop=True)
print(family_df.head(10)) 

           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4


Удалим из исходного датафрейма столбцы education и family_status.

In [27]:
df = df.drop(['education', 'family_status'], axis=1)
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32.0,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27.0,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43.0,0,0,F,компаньон,0,240525,операции с жильем
7,0,152,50.0,1,0,M,сотрудник,0,135823,образование
8,2,6929,35.0,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41.0,1,0,M,сотрудник,0,144425,покупка жилья для семьи


### Шаг 2.6. Категоризация дохода.

Cоздадим столбец total_income_category с категориями:0–30000 — 'E';30001–50000 — 'D';50001–200000 — 'C';200001–1000000 — 'B';1000001 и выше — 'A'.

Создадим функцию income. На вход функции попадает ежемесячный доход, а возвращает функция категорию клиента. Опишем функцию:

In [28]:
def income(total_income):
    if 30000 >= total_income >= 0:
        return 'E'
    if 50000 >= total_income >= 30001:
        return 'D'
    if 200000 >= total_income >= 50001:
        return 'C'
    if 1000000 >= total_income >= 200001:
        return 'B'
    if  total_income > 1000000:
        return 'A'   

Создадим отдельный столбец с категориями дохода.

In [29]:
df['income_group'] = df['total_income'].apply(income)
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,income_group
0,1,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926,27.0,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879,43.0,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152,50.0,1,0,M,сотрудник,0,135823,образование,C
8,2,6929,35.0,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188,41.0,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


Выведем статистику:

In [30]:
df['income_group'].value_counts()

C    16015
B     5041
D      350
A       25
E       22
Name: income_group, dtype: int64

Таким образом, больше всего клиентов категории С с ежемесячным доходом 50001–200000 руб.

### Шаг 2.7. Категоризация целей кредита.

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:'операции с автомобилем','операции с недвижимостью','проведение свадьбы','получение образования'.

Изучим данные в столбце purpose.

In [31]:
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Пропишем функцию.

In [32]:
def purpose_group(purpose):
    if "образов" in purpose:
        return 'получение образования'
    if "свадьб" in purpose:
        return 'проведение свадьбы'
    if "автомобил" in purpose:
        return 'операции с автомобилем'
    return 'операции с недвижимостью'

Добавим столбец с категорией цели кредита.

In [33]:
df['purpose_group'] = df['purpose'].apply(purpose_group)
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,income_group,purpose_group
0,1,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926,27.0,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879,43.0,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152,50.0,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929,35.0,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188,41.0,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [40]:
#Построим сводную таблицу
children_debt = df.pivot_table(index='children',  values='debt', aggfunc = ['count', 'sum', 'mean'])
children_debt

,count,sum,mean
,debt,debt,debt
children,,,
0,14090,1063,0.075444
1,4855,445,0.091658
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


##### Вывод 1:

Зависимость между количеством детей и возвратом кредита в срок есть:

*Из 14090 клиента без детей, задолжность по кредиту была у 1063 (7,5%). 

*Из 4855 клиента с 1 ребенком, задолжность по кредиту была у 445 (9,2%).

*Из 2128 клиента с 2 детьми, задолжность по кредиту была у 202 (9,5%) 

*Из 330 клиентов с 3 детьми, задолжность по кредиту была у 27 (8,2%) 

*Из 41 клиентов с 4 детьми, задолжность по кредиту была у 4 (9,7%) 

*У 9 клиентов с 5 детьми задолженности не было.

Так, наиболее надежными клиентами, с наименьшим количеством случаев просрочки кредита, являются клиенты без детей. У многодетных семей процент невозврата кредита в срок выше на 0,7% в случае с семьями с 3 детьми, и на 0,5% выше в случае с семьями с 4-5 детьми (всего 50 клиентов с 4-5 детьми, 4/50 = 0,08). Наиболее рискованной группой  для банка являются клиенты с 1-2 детьми.

Это, возможно, связано с тем, что клиенты без детей имеют намного меньше обязательных трат, связанных с содержанием детей. Многодетных семей в целом меньше, и к получению кредита они подходят  более осознанно, ответственно. 

##### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [41]:
#Построим сводную таблицу
family_status_debt = df.pivot_table(index='family_status_id',  values='debt', aggfunc = ['count', 'sum', 'mean'])
family_status_debt

,count,sum,mean
,debt,debt,debt
family_status_id,,,
0,12339,931,0.075452
1,4150,388,0.093494
2,959,63,0.065693
3,1195,85,0.071130
4,2810,274,0.097509


##### Вывод 2:

Зависимость между семейным положением и возвратом кредита в срок есть. Клиенты банка, состоящие в официальных отношениях как в настоящее время (семейный статус "женат/замужем"), так и в прошлом (семейный статус "вдовец / вдова", "в разводе") имеют более низкий процент невозврата кредита в срок (6,6% - 7,5%). Клиенты, не состоящие в отношениях(семейный статус "не женат / не замужем"), а также клиенты не оформившие брак официально ("гражданский брак") имеют более высокий процент невозврата кредита в срок (9,3% - 9,7%).

Возможно, это связано с тем, что  клиенты когда-либо сосотоящие в официальных отношениях, более ответственно, взвещенно подходят к получению кредита. 

Из 12339 клиентов с семейным статусом "женат/замужем" у 931 была задолжность по кредиту(7,5%). 

Из 4150 клиентов со статусом "гражданский брак" у 388 была задолжность по кредиту(9,3%).

Из 959 клиентов со статусом "вдовец / вдова" у 63 была задолжность по кредиту(6,6%).

Из 1195 клиентов со статусом "в разводе" у 85 была задолжность по кредиту(7,1%).

Из 2810 клиентов со статусом "не женат / не замужем" у 274 была задолжность по кредиту(9,7%).

##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [42]:
#Построим сводную таблицу
income_group_debt = df.pivot_table(index='income_group',  values='debt', aggfunc = ['count', 'sum', 'mean'])
income_group_debt

,count,sum,mean
,debt,debt,debt
income_group,,,
A,25,2,0.080000
B,5041,356,0.070621
C,16015,1360,0.084920
D,350,21,0.060000
E,22,2,0.090909


##### Вывод 3:

Зависимоть между уровнем дохода и возвратом кредита в срок также есть. 
Исходя их анализа наиболее многочисленных групп (группы В и С), можно сделать вывод, что самыми  надежными клиентами для банка являются люди  с высокими доходами (группа В, доход 201тыс-1млн руб.). Наиболее рискованной и в тоже время самой многочисленной группой заемщиков, являются клиенты категории С. Это люди среднего класса, с доходами 50тыс-200тыс руб. Возможно, это связано с тем, что данные клиенты не всегда рационально подходят к оформлению кредита ("переоценили свои силы"), либо с непредвиденными обстоятельствами, такими как потеря работы например.

##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

In [43]:
#Построим сводную таблицу
purpose_group_debt = df.pivot_table(index='purpose_group',  values='debt', aggfunc = ['count', 'sum', 'mean'])
purpose_group_debt

,count,sum,mean
,debt,debt,debt
purpose_group,,,
операции с автомобилем,4306,403,0.093590
операции с недвижимостью,10811,782,0.072334
получение образования,4013,370,0.092200
проведение свадьбы,2323,186,0.080069


##### Вывод 4:

Наименьшие процент клиентов (от общего числа клиентов в данной категории) с задолжностью по кредиту в категории "Операции с недвижимость" и "Проведение свадьбы" (7,2% и 8% соответственно). Так как кредит на недвижимость чаще всего берется на длительный срок и составляет значительную сумму, клиенты более осознанно подходят к его получению, к тому же недвижимость чаще всего находится в залоге у банка до момента погащения кредита и в случае длительной просрочки может быть изьята банком. Погащение кредита на проведение свадьбы, возможно, происходит с помощью родственников, полученных в подарок денежных средств.

Наибольший процент клиентов (от общего числа клиентов в данной категории) с задолжностью по кредиту в категории "Операции с автомобилем" и "Получение образования" (9,3% и 9,2% соответственно).
Получение образования длительный процесс и окупается не быстро. Покупка автомобиля влечет за собой существенные дополнительные расходы на содержание, к чему многие клиенты не готовы.

##### Общий  вывод:

В результате анализа удалось выявить зависимость возврата кредита в срок от цели кредита, уровня дохода и семейного положения клиента, а также от количества детей у клиента.